In [1]:
import os
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

import warnings
warnings.filterwarnings('ignore')

In [2]:
material_data = pd.read_csv("../../../data/masked_data/masked_material_data.csv")
trips_data = pd.read_csv("../../../data/masked_data/masked_total_trip_data.csv")

outpath = "../../../answers/set2/"

In [3]:
print(material_data.shape)
print(trips_data.shape)

(73864, 14)
(60644, 29)


In [4]:
trips_data["plan_shipment_start_date_time"] = pd.to_datetime(trips_data["plan_shipment_start_date_time"],errors='coerce')
trips_data["pick_start_date_time"] = pd.to_datetime(trips_data["pick_start_date_time"],errors='coerce')
trips_data["plan_delivery_date"] = pd.to_datetime(trips_data["plan_delivery_date"],errors='coerce')
trips_data["unload_date_time"] = pd.to_datetime(trips_data["unload_date_time"],errors='coerce')
material_data["order_created_on"] = pd.to_datetime(material_data["order_created_on"],errors='coerce')
trips_data["load_completed_date_time"] = pd.to_datetime(trips_data["load_completed_date_time"],errors='coerce')
trips_data["pick_trackingdeparttime"] = pd.to_datetime(trips_data["pick_trackingdeparttime"],errors='coerce')
trips_data["pick_end_date_time"] = pd.to_datetime(trips_data["pick_end_date_time"],errors='coerce')
trips_data["drop_trackingarrivaltime"] = pd.to_datetime(trips_data["drop_trackingarrivaltime"],errors='coerce')


In [5]:
cols = ['plan_shipment_start_date_time', 'pick_start_date_time','pick_end_date_time', 'load_completed_date_time',
        'pick_trackingeta', 'pick_trackingdeparttime','drop_trackingarrivaltime','drop_trackingeta','unload_date_time']
for col in cols:
    trips_data[col] = pd.to_datetime(trips_data[col], errors='coerce')
    trips_data[col] = np.where(trips_data[col].dt.year<2020, np.datetime64('NaT'), trips_data[col])


cols = ['order_created_on','plan_delivery_date']
for col in cols:
    material_data[col] = pd.to_datetime(material_data[col], errors='coerce')
    material_data[col] = np.where(material_data[col].dt.year<2020, np.datetime64('NaT'), material_data[col])

In [6]:
# trips_data["invoice_update"] = np.where(trips_data["order_created_on"].isna(),0,1)
trips_data["invoice_update"] = 1
trips_data["invoice_update"] = np.where(trips_data["pick_start_date_time"].isna(),trips_data["invoice_update"],2)
trips_data["invoice_update"] = np.where(trips_data["pick_end_date_time"].isna(),trips_data["invoice_update"],3)
trips_data["invoice_update"] = np.where(trips_data["load_completed_date_time"].isna(),trips_data["invoice_update"],4)
trips_data["invoice_update"] = np.where(trips_data["pick_trackingdeparttime"].isna(),trips_data["invoice_update"],5)
trips_data["invoice_update"] = np.where(trips_data["drop_trackingarrivaltime"].isna(),trips_data["invoice_update"],6)
trips_data["invoice_update"] = np.where(trips_data["unload_date_time"].isna(),trips_data["invoice_update"],7)

stage_dict = {#0: "Order not created",
              1: "Order created",
              2: "Loading started",
              3: "Loading Ended",
              4: "In Yard",
              5: "On Road",
              6: "Arrived",
              7: "Unloaded"
              }

trips_data['stage'] = trips_data["invoice_update"].map(stage_dict)

In [7]:
trips_data['stage'].value_counts()

Unloaded           54154
Order created       2533
In Yard             1733
Arrived             1303
Loading started      598
On Road              276
Loading Ended         47
Name: stage, dtype: int64

In [ ]:
material_data.to_csv("../../../data/masked_data/masked_material_data_upd.csv", index=False)
trips_data.to_csv("../../../data/masked_data/masked_total_trip_data_upd.csv", index=False)


q1: How many trips are in loading stage currently at "SALEM OR DC"?

In [34]:
out = len(trips_data[(trips_data['source_location_name']=="SALEM OR DC") & (trips_data['stage']=="Loading started")])
out

4

q2: How many trips are in ready to dispatch currently at SALEM OR DC?

In [35]:
out = len(trips_data[(trips_data['source_location_name']=="SALEM OR DC") & (trips_data['stage']=="Loading Ended")])
out

0

q3: Where is the shipment 7601987451